In [1]:
import numpy as np
import pandas as pd

# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")
import math

/var/folders/lq/0q3fzmb57s3c187k6gjyx3_00000gn/T/ipykernel_10209/852975974.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [63]:
continental_states = ['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
           'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [64]:
states = ['MI', 'WA','TX', 'FL']

In [65]:
date_cutoff = '2022-12-31'

In [120]:
def cleaned_state (state, normalize=None):
    '''Input: 2-letter state abbreviation
    Ouput: dataframe for the state that cleaned and columns paired down, and normalized if desired'''
    df = pd.read_csv(f'../Cleaning/Plant_Location_{state}.csv')
    df['period'] = pd.to_datetime(df['period'], format='%Y-%m').dt.to_period(freq="D")
    df= df.dropna(axis=0)
    
    df.rename(columns={"total-consumption": "consumption"}, inplace=True)
    
    df = df[['state','plantCode', 'period','consumption', 'Longitude', 'Latitude']]
    
    #issue with FL - some consumption numbers are "Natural Gas = 815.8 MW" (found with try: int(x), except: print)
    #All non-numbers filled in with 0s...
    df['consumption'] = pd.to_numeric(df['consumption'], errors='coerce')

    df[['Longitude','Latitude','consumption']] = df[['Longitude','Latitude','consumption']].fillna(0)
    df['plantCode'] = df['plantCode'].astype(np.int64)
    
    if normalize:
        df['consumption']= df['consumption'] / df[['consumption','plantCode']].groupby('plantCode').transform('sum')['consumption']
    
    return df

In [27]:
def multi_state_DF (states, normalize=None):
    '''Input: list of state abbreviations
    Output: Merged dataframe of all the states: with time periods as index and ST_plantcodes as columns'''
    for state in states:
        temp = cleaned_state(state, normalize)
        try:
            df = pd.concat([df,temp], axis=0)
        except:
            df = temp
    
    return df

In [80]:
def pivoted_df (df, dezero=None):
    '''Input df with data for all the states, and whether time series with 0 values should be dropped
    Ouput: dataframe for the state that is formated with time periods as index and plantCodes as columns'''
    df = df[['plantCode', 'period','consumption']]

    df = pd.pivot_table(df, values='consumption', index=['period'], columns=['plantCode'])
    
    #get rid of columns (plants)that have 0 values?
    #df = df.replace(0.0,np.nan)

    if dezero:
        df = df.replace(0.0,np.nan)
        df = df.dropna(axis=1)

    #Drop data for 2023!
    for row in df.index:
        if row >= pd.to_datetime(date_cutoff).to_period(freq="D"):
            df.drop(row, axis=0, inplace=True)
    
    return df

In [67]:
def calc_pairwise_dtw_cost(x, y, ret_matrix=False):
    """
    Takes in two series. If ret_matrix=True, returns the full DTW cost matrix;
    otherwise, returns only the overall DTW cost
    """
    cost_matrix = np.zeros((len(x), len(y)))
    dtw_cost = None
    dist_fn = lambda a, b: (a - b) ** 2  # Optional helper function
    
    for i in range(len(x)): #x = i = row
        for j in range(len(y)): # y = j = col
        #If the coordinate is (0,0): define the cost matrix as d(x_(i),y_(j))
            if i==0 and j==0:
                cost_matrix[i,j]=dist_fn(x[i],y[j])
        #Otherwise;
            else:
            #the cost matrix is equal to the sum of d(x_(i),y_(j)) and the minimum of the following:
    #                         1)  If the row is less than or equal to zero;
                if (i-1) < 0:
                    up = math.inf
                else:
                    up = cost_matrix[j, i -1]
    #                                      * compute the cost matrix using i and j-1
    #                              Else: too large to compute
    #                          2)  If the column is less than or equal to zero;
                if (j-1) < 0:
                    left = math.inf
                else:
                    left = cost_matrix[j - 1, i]
    #                            * compute the cost matrix using i-1 and j
    #                              Else: too large to compute
    #                          3)  If the row and columns are less than or equal to zero;
                if (j-1) >= 0 and (i-1) >= 0:
                    diag =  cost_matrix[j-1, i-1] # i ==0, j==0 case already handled
        #                                    * too large to compute
                else:
                    diag = math.inf
                cost = min(up,left,diag)

                cost_matrix[j,i] = dist_fn(x[i],y[j]) + cost
    dtw_cost = cost_matrix[-1][-1]
    
    return cost_matrix if ret_matrix else dtw_cost

In [12]:
def calc_dtw_cost(df):
    """
    Takes in a DataFrame and computes all pairwise DTW costs
    """
    
    dtw_cost_df = pd.DataFrame(np.zeros((len(df.columns),len(df.columns))),
                              index=df.columns,
                              columns=df.columns)
    
    for i in range(len(df.columns)):
        for j in range(len(df.columns)):
            if i == j:
                pass
            else:
                dtw_cost_df.iloc[i,j] = calc_pairwise_dtw_cost(df.iloc[:,i], df.iloc[:,j])
    
    return dtw_cost_df

In [13]:
def DTW_distance_matrix (df, output='DTW_Matrix.csv'):
    '''Inupt: list of state abbreviations, tag to normalize or not
    Output: Dynamic Time Warp distance matrix for all plants in states list'''

    DTW = calc_dtw_cost(df)
    
    #Handy if computing large matrix, time intensize to run
    DTW.to_csv(output, index=False)
    
    return DTW    

In [115]:
def merge_and_DTW (states, normalize=None, output=None):
    '''Input: state - a list of states
        normalize - if True, then the consumption values for each plant will be normalized
        dezero - if True, then removes all plants that have a value of 0 consumption at some point
        output - the file name signifier for the output file
    Output: saves specified DTW distance matrix and merged datafile'''
    
    raw_df = multi_state_DF (states, normalize=normalize)
    
    df = pivoted_df(raw_df, dezero=None)
    #DTW_distance_matrix (df, output=f'DTW_Matrix_{output}.csv')
    
    timeseries = df.T
    timeseries.reset_index(inplace=True)
    dict_df = raw_df[['state', 'plantCode','Longitude', 'Latitude']].drop_duplicates()
    merge = pd.merge(dict_df, timeseries, how='left', left_on='plantCode', right_on='plantCode')
    merge = merge.fillna(0.0)

    merge.to_csv(f'Merge_{output}.csv', index=False)
    
    #df = pivoted_df(raw_df, dezero=True)
    #DTW_distance_matrix (df, output=f'DTW_Matrix_{output}_No0.csv')
    
    return print(f'Success for {output}')
        

In [31]:
def create_merge_DTWs (states):
    merge_and_DTW(states, output='Raw')
    merge_and_DTW(states, normalize=True, output='Norm')
    
    return print(f'Total success for {states}')

In [121]:
create_merge_DTWs (states)

Success for Raw
Success for Norm
Total success for ['MI', 'WA', 'TX', 'FL']


In [122]:
df = pd.read_csv('Merge_Raw.csv')
len(df), len(df.columns)

(503, 52)